In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow 

from pprint import pprint
from IPython.display import display

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import yaml
from dotenv import load_dotenv


def load_config(config_path="./configs/config.yaml"):
    with open(config_path, "r") as file:
        config = yaml.safe_load(file)
    return config


load_dotenv()

config = load_config()
print(config)

{'system_prompt_name': 'papar2notion_system', 'system_prompt_num': 1, 'human_prompt_name': 'papar2notion_human', 'human_prompt_num': 1}


In [17]:
url = f'{os.getenv("MLFLOW_TRACKING_URI")}'
print(url)

http://localhost:30001


In [18]:
import mlflow

uri = mlflow.get_tracking_uri()

# 실험 가져오기 또는 생성
exp = mlflow.get_experiment_by_name("test-connection")
if exp is None:
    exp_id = mlflow.create_experiment("test-connection")
    print("Created experiment ID:", exp_id)
else:
    print("Experiment exists, ID:", exp.experiment_id)

# 테스트 로그
with mlflow.start_run():
    mlflow.log_param("test_param", "ok")
print("✅ 연결 및 로그 완료")

Experiment exists, ID: 1
🏃 View run adventurous-yak-763 at: http://localhost:30001/#/experiments/0/runs/4624360a22194923a6339797928ca294
🧪 View experiment at: http://localhost:30001/#/experiments/0
✅ 연결 및 로그 완료


In [3]:
# import google.generativeai as genai

# # Gemini API 키 입력


# def list_gemini_models():
#     """
#     Google Gemini API로 사용 가능한 모델 이름 출력
#     """
#     # API 키로 인증
#     genai.configure()

#     # 사용 가능한 모델 목록 가져오기
#     models = genai.list_models()

#     print("📦 사용 가능한 Gemini 모델 목록:")
#     for model in models:
#         print(f"- {model.name}")


# # 실행
# if __name__ == "__main__":
#     list_gemini_models()

In [19]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage


def create_llm(model_type: str = "gemini", **kwargs):
    """
    LLM 인스턴스 생성

    Args:
        model_type: "gemini" 또는 "openai"
        **kwargs: 모델별 추가 설정
    """
    if model_type.lower() == "gemini":
        print("Using Google Gemini model")
        return ChatGoogleGenerativeAI(
            model=kwargs.get("model", "gemini-2.5-pro"),
            temperature=kwargs.get("temperature", 0.0),
        )

    elif model_type.lower() == "openai":
        print("Using OpenAI model")
        return ChatOpenAI(
            model=kwargs.get("model", "gpt-4o-mini"),
            temperature=kwargs.get("temperature", 0.0),
        )

    else:
        raise ValueError(f"지원하지 않는 모델 타입: {model_type}")

In [20]:
llm = create_llm("gemini")

Using Google Gemini model


In [ ]:
import mlflow
from pydantic import BaseModel, Field
from typing import List, Optional
import datetime
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate


class Output(BaseModel):
    title: str = Field(..., description="논문의 제목")
    year: int = Field(..., description="논문의 출판 연도")
    citation: int = Field(..., description="논문의 인용 수")
    short_summary: str = Field(..., description="논문의 요약")
    tags: List[str] = Field(
        ...,
        description="논문의 키워드. 키워드는 논문을 분류하기 위해 필요한 정보. 대분류부터 디테일한 키워드까지 포함. 최대 4개.",
    )
    content: List[str] = Field(
        ...,
        description="논문의 주요 내용. Content의 주요 내용으로 system prompt에서 지시한 내용의 답변입니다.",
    )
    paper_url: str = Field(..., description="논문의 URL")


class LLMChain:
    def __init__(
        self,
        experiment_name: str = "paper2notion",
        model_type: str = "gemini",
        prompt_name: str = "paper2notion",
        system_prompt_num: int = 1,
        human_prompt_num: int = 1,
        **kwargs,
    ):
        print(f"model type: {model_type}")
        self.llm = self.create_llm(model_type, **kwargs)
        self.system_prompt, self.human_prompt = self.get_prompt(
            prompt_name=prompt_name,
            system_prompt_num=system_prompt_num,
            human_prompt_num=human_prompt_num,
        )
        self.experiment_id, self.run_name = self.create_run_name(
            experiment_name, model_type, system_prompt_num, human_prompt_num
        )
        self.__setup__(self.llm, self.system_prompt, self.human_prompt)

    def create_run_name(
        self, experiment_name, model_type, system_prompt_num, human_prompt_num
    ):
        if experiment := mlflow.get_experiment_by_name(experiment_name):
            experiment_id = experiment.experiment_id
        else:
            experiment_id = mlflow.create_experiment(experiment_name)
        run_name = datetime.datetime.now().strftime("%m%d-%H%M%S")
        run_name += f"_{model_type}_s{system_prompt_num}_h{human_prompt_num}"
        return experiment_id, run_name

    def create_llm(self, model_type: str = "gemini", **kwargs):
        if model_type.lower() == "gemini":
            return ChatGoogleGenerativeAI(
                model=kwargs.get("model", "gemini-2.5-pro"),
                temperature=kwargs.get("temperature", 0.0),
            )
        elif model_type.lower() == "openai":
            return ChatOpenAI(
                model=kwargs.get("model", "gpt-4o-mini"),
                temperature=kwargs.get("temperature", 0.0),
            )
        else:
            raise ValueError(f"지원하지 않는 모델 타입: {model_type}")

    def get_prompt(self, prompt_name, system_prompt_num=1, human_prompt_num=1):
        system_prompt = mlflow.genai.load_prompt(  # type: ignore
            f"prompts:/paper2notion_system/{system_prompt_num}"
        ).to_single_brace_format()
        human_prompt = mlflow.genai.load_prompt(  # type: ignore
            f"prompts:/paper2notion_human/{human_prompt_num}"
        ).to_single_brace_format()

        print(f"System Prompt {system_prompt_num}:")
        print(system_prompt)

        print(f"Human Prompt {human_prompt_num}:")
        print(human_prompt)

        return system_prompt, human_prompt

    def __setup__(self, llm, system_prompt, human_prompt):
        parser = PydanticOutputParser(pydantic_object=Output)
        format_instructions = (
            parser.get_format_instructions().replace("{", "{{").replace("}", "}}")
        )

        prompt = ChatPromptTemplate.from_messages(
            [("system", system_prompt), ("human", human_prompt + format_instructions)]
        )

        self.chain = prompt | llm | parser

    def run(self, paper_url):
        input_dict = {"paper_url": paper_url}

        with mlflow.start_run(experiment_id=self.experiment_id, run_name=self.run_name):
            mlflow.log_param("paper_url", paper_url)
            output = self.chain.invoke(input_dict)
            mlflow.log_params(output.dict())
            return output


chain = LLMChain(
    experiment_name="paper2notion",
    model_type="gemini",
    system_prompt_num=4,
    human_prompt_num=2,
)

output = chain.run("https://arxiv.org/pdf/2305.07895")

model type: gemini
System Prompt 4:
## System
너는 설명을 친절하게 잘해주는 교수야. 
논문을 아래와 같이 요약해서 배경지식이 없는 학생들에게 설명한다고 생각해.
논문을 읽기 전 미리 배경지식을 얻을 수 있는 요약을 만들어줘.
반드시 content 주요 내용 사이에는 "---"을 삽입해서 명확히 구분해줘.

## Content 주요 내용
아래와 같이 9가지 부분으로 요약해줘.
1. 연구 동기(왜 이 연구를 했는지)와 논문 제목의 이유.
2. 요약. 이 부분은 Abstract과 Conclusion 부분의 내용을 요약.
    요약할 때는 왜 이 연구를 했는지(Why), 그래서 어떤 연구를 제시했는지(What), 어떻게 적용 했는지(How)로 설명해줘.
3. 논문의 methodology. 직관적인 예시를 통해 자세한 설명 부탁해. 
4. 논문에서 사용한 데이터 셋(dataset)과 평가 지표(evaluation metric)에 대해 설명해줘.
    - 데이터 셋과 평가지표는 본문 내용을 전부 알려줘.
5. 결과에 대해 간략히 설명해줘.
6. 논문이 제시한 방법의 장단점.
7. 이 논문에서 나온 추천할 만한 References. 예를 들어, Reference를 직접적으로 발전 시켜 연구한 논문이면, reference를 부탁해.
8. 아래 정보도 개조식으로 정리해줘. 각 요소마다 복사하기 편하게 정리해줘. 예를 들어, title 따로(title만), year 따로(YYYY 숫자만), citation 따로(숫자만), tags 따로(tag1, tag2, tag3, tag4)
- title: 논문 이름
- year: 발행년도
- citation: 인용수
- tags: 논문을 분류하기 위해 필요한 키워드들, 큰 카테고리 및 세부 기술의 키워드가 필요함.
9. short_summary: 최종적으로 전체적인 내용을 핵심적인 2개 문장을 개조식으로 요약해줘. 반드시 천천히 생각해보고 차근차근 생각해서 핵심적인 단어와 표현으로 부탁해. 제일 중

In [ ]:
import requests
import os
from dotenv import load_dotenv


def parse_notion_json(output):
    title = output.title
    year = output.year
    citation = output.citation
    summary = output.short_summary
    tags = output.tags
    content = output.content
    paper_url = output.paper_url

    new_page_json = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {  # ✅ Title은 반드시 있어야 함
                "title": [{"text": {"content": title}}]
            },
            "Year": {"number": year},  # ✅ 숫자형
            "Citation": {"number": citation},  # ✅ 숫자형
            "Summary": {"rich_text": [{"text": {"content": summary}}]},  # ✅ rich_text
            "Tag": {"multi_select": [{"name": tag} for tag in tags]},  # ✅ multi_select
            "Done": {"checkbox": False},  # ✅ 체크박스
            "URL": {"url": paper_url},  # ✅ URL
            "Finish": {"checkbox": False},  # ✅ 체크박스
            # "End": {"date": {"start": "2025-07-06"}},  # ✅ 날짜
            "End": {"date": None},  # ✅ 날짜
        },
        "children": [  # ✅ 본문 내용 추가
            {
                "object": "block",
                "type": "heading_2",
                "heading_2": {
                    "rich_text": [
                        {"type": "text", "text": {"content": "Paper Content"}}
                    ],
                },
            },
            {
                "object": "block",
                "type": "paragraph",
                "paragraph": {
                    "rich_text": [
                        {"type": "text", "text": {"content": content_item}}
                        for content_item in content
                    ]
                },
            },
        ],
    }

    return new_page_json


# 환경변수 또는 직접 입력
NOTION_API_KEY = os.getenv("NOTION_API_KEY")  # 노션 Integration API 키
DATABASE_ID = os.getenv("PAPER_DATABASE_ID")  # 데이터베이스 ID
NOTION_VERSION = "2022-06-28"


# .env 로드
load_dotenv()

# 환경변수 체크
if not NOTION_API_KEY or not DATABASE_ID:
    raise ValueError(
        "❌ NOTION_API_KEY 또는 PAPER_DATABASE_ID 환경변수가 비어있습니다."
    )

new_page_json = parse_notion_json(output)

# API 요청
response = requests.post(
    "https://api.notion.com/v1/pages",
    headers={
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Notion-Version": NOTION_VERSION,
        "Content-Type": "application/json",
    },
    json=new_page_json,
)

# 결과 출력
if response.status_code in [200, 201]:
    print("✅ 페이지가 성공적으로 생성되었습니다!")
    print(response.json())
else:
    print("❌ 오류 발생:", response.status_code)
    print(response.text)

["1. 연구 동기(왜 이 연구를 했는지)와 논문 제목의 이유.\n안녕하세요, 학생 여러분. 오늘 다룰 논문은 LLM(거대 언어 모델)이 어떻게 스스로 더 똑똑해질 수 있는지에 대한 흥미로운 연구입니다. 우리가 글을 쓸 때 초고를 작성하고, 다시 읽어보며 어색한 부분을 고치는 것처럼, LLM도 한 번에 완벽한 결과물을 내놓기는 어렵습니다. 이 연구는 바로 이 지점에서 출발합니다. LLM이 생성한 결과물을 스스로 평가하고, 그 피드백을 바탕으로 더 나은 결과물을 만들도록 할 수는 없을까? 라는 질문이 연구의 동기입니다. 논문 제목인 'Self-Refine'은 이 과정을 아주 직관적으로 보여줍니다. 모델 스스로(Self) 자신의 결과물을 개선(Refine)한다는 의미를 담고 있죠. 외부의 도움 없이 스스로 발전하는 똑똑한 모델을 만들고자 한 것입니다.", "2. 요약. 이 부분은 Abstract과 Conclusion 부분의 내용을 요약.\n이 논문을 '왜, 무엇을, 어떻게'로 요약해 보겠습니다. (Why) 왜 이 연구를 했을까요? 앞서 말했듯, GPT와 같은 LLM은 한 번의 시도로는 복잡한 문제에 대해 부정확하거나 미흡한 답변을 내놓는 경우가 많기 때문입니다. (What) 그래서 어떤 연구를 제시했나요? 연구진은 'Self-Refine'이라는 간단하지만 강력한 프레임워크를 제안했습니다. 이는 LLM이 자신의 결과물을 비평하고, 그 비평을 바탕으로 스스로 결과물을 수정해 나가는 반복적인 과정입니다. (How) 어떻게 적용했을까요? 별도의 모델 훈련이나 데이터 없이, 이미 존재하는 LLM(예: GPT-3.5, GPT-4)에게 특정 역할을 부여하는 프롬프트(지시어)만으로 이 과정을 구현했습니다. 코딩, 작문, 수학 문제 풀이 등 다양한 작업에 적용하여 Self-Refine이 기존의 단일 답변 방식보다 훨씬 뛰어난 성능을 보임을 입증했습니다.", '3. 논문의 methodology. 직관적인 예시를 통해 자세한 설명 부탁해.\nSelf-Refine의 방법론은 우리가 과